In [1]:
import numpy as np
import re
import matplotlib as mpl
import matplotlib.pyplot as plt

import pyparsing as pp
from pyparsing import pyparsing_common as ppc
import seaborn as sns

import pandas as pd

from tqdm import tqdm

5 edgerunners are on top of arasaka tower. They are bleeding out having been shot by arasaka security forces. Arasaka security forces are still there. Only 1 of them has TT membership. Oh and there is a blidning thunderstorm with rain obscuring visibility.

In [ ]:
f'{number}{who}{where}. {injury} having been {danger}. {Danger} is/are still present. {x} of them has/have TT membership. Oh and {modifier}.'

In [ ]:

# must combine person and number. they should always come together. whenver person is pulled the number should always be calculated with it.
who = 'Edgerunner(s)'
number = np.round(np.exp(np.abs(np.random.randn())))

injury = "lost limbs"
injury_urgency = 'high'
location = 'Arasaka tower'

# write a function that creates a chance of having several injuries
threat = 'Arasaka strike team'
threat_present = True
tt_memberships = min(np.random.geometric(p=0.9),number)
tt_cover_type = 'Silver'

modifier = 'severe thunderstorm'

In [ ]:
print(
f"""Who?\t\t{who}
How many?\t{number:n}
TT memberships\t{tt_memberships:n}
Injury\t\t{injury}
Injury urgency\t{injury_urgency}
Where?\t\t{location}
Threat\t\t{threat}
Threat present?\t{'Yes' if threat_present else 'No'}
Modifier\t{modifier}
""")

- If the top row of the random table input does not have any generators then it is used as a title of the table
- if any subsequent row does not have any generators the text is just output as is.
- look into inputting file when running python from terminal
- ~~define vlo/lo/med/hi/vhi distributions~~
- in the class have default values for the distributions but allow for people to input their own upon init


- make process input file a method that has to be called separately and not upon __init__
- after that retry the dictionary method of process input line
- add an error handling for no table found - try, else, finally
- create a tool that inspects all the lines in all the tables and surfaces the lines that have errors in them.

- add more ways to do number generation. it shouldn't be just the geometric distributino.
    - add uniform 0-9 so that any number can be generated

In [342]:
class Random_Table_Generator:
    

    
    
    def __init__(
        self,
        distribution_modifier=1,
        tables_folder = 'tables',
        input_phrase = None,
        default_chance_increase = 2,
        default_chance_decrease = 0.5,
        default_option = 0.5,
        default_boolean = 0.5,
        loop_limit = 100,
        dist_dict = {
            'vlo': 0.9,
            'lo':  0.7,
            'med': 0.3,
            'hi':  0.2,
            'vhi': 0.1
        }
    ):
        self.distribution = distribution_modifier
        self.default_chance_increase = default_chance_increase
        self.default_chance_decrease = default_chance_decrease
        self.default_option = default_option
        self.default_boolean = default_boolean
        self.tables_folder = tables_folder
        self.input_phrase = input_phrase
        self.loop_limit = loop_limit
        self.dist_dict = dist_dict
        
        # defining patterns for pyparsing
        self.option_pat_combo    = '%' + pp.Opt(ppc.fnumber()) + pp.nested_expr(opener='[', closer=']')
        self.option_pat_combo2   = pp.original_text_for('%' + pp.Opt(ppc.fnumber()('chance')) + '.' + pp.original_text_for(pp.nested_expr(opener='[', closer=']'))('heads') + pp.Opt('.') + pp.original_text_for(pp.Opt((pp.nested_expr(opener='[', closer=']'))))('tails'))
        self.option_pat_indiv_orig    = '%' + pp.Opt(ppc.fnumber()('chance')) + '.' + pp.original_text_for(pp.nested_expr(opener='[', closer=']'))('heads') + pp.Opt('.') + pp.original_text_for(pp.Opt(pp.nested_expr(opener='[', closer=']')))('tails')
        self.option_pat_indiv    = '%' + pp.Opt(ppc.fnumber()('chance')) + '.' + pp.original_text_for(pp.nested_expr(opener='[', closer=']'))('heads') + pp.Opt('.') + pp.original_text_for(pp.Opt(pp.nested_expr(opener='[', closer=']')))('tails') 
        self.table_pat           = pp.Combine('@' + pp.Word(pp.printables, exclude_chars=']'))
        self.number_pat_combo    = pp.Combine('#' + pp.Word(pp.printables))
        self.number_pat_indiv    = '#' + pp.Word(pp.alphas)('dist') + pp.Opt('.') + pp.Opt(pp.Word(pp.nums)('max'))
        self.boolean_pat_combo   = pp.Combine('$' + pp.Opt(ppc.fnumber) + "." + pp.Word(pp.alphanums) + "." + pp.Word(pp.alphanums))
        self.boolean_pat_indiv   = '$' + pp.Opt(ppc.fnumber)('chance') + "." + pp.Word(pp.alphanums)('heads') + "." + pp.Word(pp.alphanums)('tails')
        self.chance_increase_pat = pp.Combine(pp.StringStart() + '^' + pp.Opt(ppc.fnumber()))
        self.chance_decrease_pat = pp.Combine(pp.StringStart() + '*' + pp.Opt(ppc.fnumber()))
        self.pat_dict = {
            'process_table': self.table_pat
            ,'process_boolean': self.boolean_pat_combo
            ,'process_number': self.number_pat_combo
        }

        
        
    def process_input_file(self, input_file):
        file = open(input_file, 'r').read().split('\n')
        for line in file:
            processed_line = self.process_input_line(line)
            print(processed_line)
            
            
    def process_input_phrase(self, input_phrase):
        # is this the same as process_input_line?
        pass
    
    def test_process_input_line(self, line):
        # this is for testing the compact dictionary version of process_input_line.
        pass
    
    
    def process_input_line(self, line):
        # the expanded version that works without the pat_dict
        
        gen_check = False
        outputs = []
        matches_option_pat = Random_Table_Generator.option_pat_combo2.scan_string(line)
        for match, start, stop in matches_option_pat:
            gen_check = True
            output = self.process_input_line(self.process_option(match))
            outputs.append((match, output, start, stop))
        if gen_check:
            line = self.replace(line, outputs)
        
        gen_check = False
        outputs = []
        matches_table_pat = Random_Table_Generator.table_pat.scan_string(line)
        for match, start, stop in matches_table_pat:
            gen_check = True
            output = self.process_input_line(self.process_table(match))
            outputs.append((match, output, start, stop))
        if gen_check:
            line = self.replace(line, outputs)
        
        gen_check = False
        outputs = []
        matches_number_pat = Random_Table_Generator.number_pat_combo.scan_string(line)
        for match, start, stop in matches_number_pat:
            gen_check = True
            output = self.process_input_line(self.process_number(match))
            outputs.append((match, output, start, stop))
        if gen_check:
            line = self.replace(line, outputs)
        
        gen_check = False
        outputs = []
        matches_boolean_pat = Random_Table_Generator.boolean_pat_combo.scan_string(line)
        for match, start, stop in matches_boolean_pat:
            gen_check = True
            output = self.process_input_line(self.process_boolean(match))
            outputs.append((match, output, start, stop))
        if gen_check:
            line = self.replace(line, outputs)
        
        return line

    
    def replace(self, line, mappings):
        slices, indices, output = [], [0], []
        for mapping,_, start, end in mappings:
            if str(mapping[0])[-1] == ' ':
                end -= 1
            indices += [start,end]

        # if len(line) != mappings[-1][-1]:
        #     indices += [None]
        indices += [None]

        for i in range(0,len(indices),2):
            slices.append(slice(indices[i], indices[i+1]))

        if mappings[0][2] == 0:
            for new_word, i in zip([i[1] for i in mappings], slices):
                output += [new_word, line[i]]
            if len(mappings) > len(slices):
                output += [mappings[-1][1]] 
        else:
            for new_word, i in zip([i[1] for i in mappings], slices):
                output += [line[i], new_word]
            if len(mappings) < len(slices):
                output += [line[slices[-1]]] 
        return ''.join(output)

    
    def process_option(self, instruction):
        for match, start, stop in Random_Table_Generator.option_pat_indiv.scan_string(instruction):
            if match.chance == '':
                probability = default_boolean
            else:
                probability = match.chance
            random_value = np.random.random()
            
            return (match.heads[1:-1] if random_value <= probability else match.tails[1:-1])
    
    
    def process_table(self, reference):
        input_file = self.tables_folder + '/' + reference[0][1:] + '.txt'
        file = open(input_file, 'r').read().split('\n')

        probabilities = np.ones(len(file))

        for line_no, line in enumerate(file):
            matches_increase = Random_Table_Generator.chance_increase_pat.scan_string(line)
            for match, start, end in matches_increase:
                if end == 1:
                    probabilities[line_no] = self.default_chance_increase
                else:
                    probabilities[line_no] = float(match[0][1:])
                file[line_no] = line[(end+1):]

            matches_decrease = Random_Table_Generator.chance_decrease_pat.scan_string(line)
            for match, start, end in matches_decrease:
                if end == 1:
                    probabilities[line_no] = self.default_chance_decrease
                else:
                    probabilities[line_no] = float(match[0][1:])
                file[line_no] = line[(end+1):]

        probabilities_cum = np.cumsum(probabilities)
        random_value = np.random.uniform(0,probabilities.sum(),1)
        pick = np.argmax((probabilities_cum // random_value)>0)
        
        return file[pick]
    
    
    def process_number(self, instruction):
        for match, start, end in Random_Table_Generator.number_pat_indiv.scan_string(instruction):
            random_value = np.random.geometric(p=self.dist_dict[match.dist])
            if match.max == '':
                return random_value
            else:
                return (str(random_value) if int(match.max) > random_value else match.max)
    
    
    def process_boolean(self, instruction):
        for match, start, end in Random_Table_Generator.boolean_pat_indiv.scan_string(instruction):
            if match.chance == '':
                probability = self.default_boolean
            else:
                probability = match.chance
            random_value = np.random.random()
            return (match.heads if random_value <= probability else match.tails)
    
    

            
            
rtg = Random_Table_Generator()
rtg.process_input_file("TEST.txt")

control line Camels space
test1 fail
test2 friends forever or 
test3 hola beef without we are victorious soup
test4 second bracket outside the brackets
 Trauma Team 2


### Process Option Function

In [ ]:
file = open('TEST.txt', 'r').read().split('\n')

option_pat = '%' + pp.Opt(ppc.fnumber()) + pp.nested_expr(opener='[', closer=']')
table_pat = '@' + pp.Word(pp.printables)

pat_dict = {
    'option_pat': option_pat,
    'table_pat': table_pat
}

del matches

def table_pat():
    print('test successful!')

for i in pat_dict:
    locals()[i]()
    for line in file:
        print('\n\n'+ line)
        matches = pat_dict[i].scanString(line)

        for match, start, stop in matches:
            print(match, start, stop)

In [293]:
file = open('TEST.txt', 'r').read().split('\n')
table_pat = pp.Combine('@' + pp.Word(pp.printables, exclude_chars=']'))


for line in file:
    matches = table_pat.scan_string(line)
    for a,b,c in matches:
        print(type(str(a)))
        print(a,b,c)


<class 'str'>
['@animal'] 13 20
<class 'str'>
['@animal'] 49 56


In [179]:
option_pat_combo = '%' + pp.Combine(pp.Opt(ppc.fnumber()) + '.' + pp.nested_expr(opener='[', closer=']') + pp.Opt('.') + pp.Opt(pp.nested_expr(opener='[', closer=']')))
option_pat_combo2 = pp.original_text_for('%' + pp.Opt(ppc.fnumber()('chance')) + '.' + pp.original_text_for(pp.nested_expr(opener='[', closer=']'))('heads') + pp.Opt('.') + pp.Opt(pp.original_text_for(pp.nested_expr(opener='[', closer=']')))('tails'))
option_pat_indiv = '%' + pp.Opt(ppc.fnumber()('chance')) + '.' + pp.original_text_for(pp.nested_expr(opener='[', closer=']'))('heads') + pp.Opt('.') + pp.Opt(pp.original_text_for(pp.nested_expr(opener='[', closer=']')))('tails')

default_boolean = 0.5


def process_option(line):
    for match, start, stop in option_pat_indiv.scan_string(line):
        print('full match for this line is: ', match, start, stop)
        print('chance for this line is: ', match.chance)
        print('heads for this line is: ', match.heads)
        print('tails for this line is: ', match.tails)

        if match.chance == '':
            probability = default_boolean
        else:
            probability = match.chance
        random_value = np.random.random()
        print(f'{random_value=}')
        
        return (match.heads[1:-1] if random_value <= probability else match.tails[0][1:-1])

    
test_option1 = "test1 %.[success].[fail]"
test_option2 = "test2 %0.9.[friends forever] or %.[never will be @friends]"
test_option3 = "test3 %.[hola %0.9.[beef without %.[we are victorious] soup]]"
test_option4 = "test4 %.[success] . and then it goes on"

print(f'result for test_option1 is: ', process_option(test_option1), '\n')
print(f'result for test_option2 is: ', process_option(test_option2), '\n')
print(f'result for test_option3 is: ', process_option(test_option3), '\n')
print(f'result for test_option4 is: ', process_option(test_option4), '\n')

full match for this line is:  ['%', '.', '[success]', '.', '[fail]'] 6 24
chance for this line is:  
heads for this line is:  [success]
tails for this line is:  ['[fail]']
random_value=0.06223249279666143
result for test_option1 is:  success 

full match for this line is:  ['%', 0.9, '.', '[friends forever]'] 6 29
chance for this line is:  0.9
heads for this line is:  [friends forever]
tails for this line is:  
random_value=0.24710868022238064
result for test_option2 is:  friends forever 

full match for this line is:  ['%', '.', '[hola %0.9.[beef without %.[we are victorious] soup]]'] 6 61
chance for this line is:  
heads for this line is:  [hola %0.9.[beef without %.[we are victorious] soup]]
tails for this line is:  
random_value=0.05926164424516245
result for test_option3 is:  hola %0.9.[beef without %.[we are victorious] soup] 

full match for this line is:  ['%', '.', '[success]', '.'] 6 20
chance for this line is:  
heads for this line is:  [success]
tails for this line is:  
ra

In [ ]:
dumb_test = pp.nested_expr(opener='[', closer=']')


### Replace Function

In [ ]:
test_line = 'as we are going to the supermarket to buy apples'
mappings = [('we', 'they', 3, 5), ('supermarket', 'bazaar', 23, 34)]

slices = []
indices = [0]
output = []

for _,_, start, end in mappings:
    indices += [start,end]

if len(test_line) != mappings[-1][-1]:
    indices += [-1]

print(indices)

for i in range(0,len(indices),2):
    print(i)
    slices.append(slice(indices[i], indices[i+1]))

print(slices)
    
if mappings[0][2] == 0:
    for new_word, i in zip([i[1] for i in mappings], slices):
        output += [new_word, test_line[i]]
    if len(mappings) > len(slices):
        output += [mappings[-1][1]] 

else:
    for new_word, i in zip([i[1] for i in mappings], slices):
        output += [test_line[i], new_word]
    if len(mappings) < len(slices):
        output += [test_line[slices[-1]]] 

    
    
def replace(line, mappings):
    slices, indices, output = [], [0], []

    for _,_, start, end in mappings:
        indices += [start,end]

    if len(line) != mappings[-1][-1]:
        indices += [None]

    for i in range(0,len(indices),2):
        slices.append(slice(indices[i], indices[i+1]))

    if mappings[0][2] == 0:
        for new_word, i in zip([i[1] for i in mappings], slices):
            output += [new_word, line[i]]
        if len(mappings) > len(slices):
            output += [mappings[-1][1]] 
    else:
        for new_word, i in zip([i[1] for i in mappings], slices):
            output += [line[i], new_word]
        if len(mappings) < len(slices):
            output += [line[slices[-1]]] 


    return ''.join(output)

replace(test_line, mappings)

In [ ]:
def replace(line, mappings):
    slices = []
    indices = [0]
    output = []

    for _,_, start, end in mappings:
        indices += [start,end]

    if len(line) != mappings[-1][-1]:
        indices += [None]

    for i in range(0,len(indices),2):
        slices.append(slice(indices[i], indices[i+1]))

    if mappings[0][2] == 0:
        for new_word, i in zip([i[1] for i in mappings], slices):
            output += [new_word, line[i]]
        if len(mappings) > len(slices):
            output += [mappings[-1][1]] 
    else:
        for new_word, i in zip([i[1] for i in mappings], slices):
            output += [line[i], new_word]
        if len(mappings) < len(slices):
            output += [line[slices[-1]]] 


    return ''.join(output)

replace(test_line, mappings)

In [ ]:
NEW SNIP NEW SNIP NEW
NEW SNIP NEW SNIP

SNIP NEW SNIP NEW SNIP
SNIP NEW SNIP NEW


In [ ]:
[i[1] for i in mappings]

In [ ]:
a = slice(0,3)
b = slice(5,23)
c = slice(34,-1)
test_list = [test_line[a], test_line[b], test_line[c]]
test_list

In [ ]:
test_line[:None]

### Process Table Function

In [ ]:
chance_increase_pat = pp.Combine(pp.StringStart() + '^' + pp.Opt(ppc.fnumber()))
chance_decrease_pat = pp.Combine(pp.StringStart() + '*' + pp.Opt(ppc.fnumber()))

default_chance_increase = 2
default_chance_decrease = 0.5

def process_table(reference):
    input_folder = 'tables'
    input_file = input_folder + '/' + reference[1:] + '.txt'
    file = open(input_file, 'r').read().split('\n')
    
    probabilities = np.ones(len(file))

    for line_no, line in enumerate(file):
        matches_increase = chance_increase_pat.scan_string(line)
        for match, start, end in matches_increase:
            if end == 1:
                probabilities[line_no] = default_chance_increase
            else:
                probabilities[line_no] = float(match[0][1:])
            file[line_no] = line[(end+1):]
        
        matches_decrease = chance_decrease_pat.scan_string(line)
        for match, start, end in matches_decrease:
            if end == 1:
                probabilities[line_no] = default_chance_decrease
            else:
                probabilities[line_no] = float(match[0][1:])
            file[line_no] = line[(end+1):]
    

    probabilities_cum = np.cumsum(probabilities)
    random_value = np.random.uniform(0,probabilities.sum(),1)
    pick = np.argmax((probabilities_cum // random_value)>0)
    
    return file[pick]
    
    
test_reference = '@corporation'
process_table(test_reference)

In [ ]:
result = []
for i in tqdm(range(10000)):
    result.append(process_table(test_reference))

In [ ]:
ax = sns.displot(data=result)
ax.set_xticklabels(rotation=90)
plt.show()

### Process Boolean Function

In [ ]:
boolean_pat_combo = pp.Combine('$' + pp.Opt(ppc.fnumber) + "." + pp.Word(pp.alphanums) + "." + pp.Word(pp.alphanums))
boolean_pat_indiv = '$' + pp.Opt(ppc.fnumber)('chance') + "." + pp.Word(pp.alphanums)('heads') + "." + pp.Word(pp.alphanums)('tails')
default_boolean = 0.5

def process_boolean(instruction):
    for match, start, end in boolean_pat_indiv.scan_string(instruction):
        if match.chance == '':
            probability = default_boolean
        else:
            probability = match.chance
        random_value = np.random.random()
        return (match.heads if random_value <= probability else match.tails)
    




test_instruction1 = '$.Yes.No'
test_instruction2 = '$0.8.Success.Fail'

print(process_boolean(test_instruction1))
print(process_boolean(test_instruction2))

In [ ]:
np.random.random()

### Process Number Function

In [ ]:
number_pat_indiv    = '#' + pp.Word(pp.alphas)('dist') + pp.Opt('.') + pp.Opt(pp.Word(pp.nums)('max'))

dist_dict = {
    'vlo': 0.9,
    'lo':  0.7,
    'med': 0.3,
    'hi':  0.2,
    'vhi': 0.1
}

def process_number(instruction):
    for match, start, end in number_pat_indiv.scan_string(instruction):
        random_value = np.random.geometric(p=dist_dict[match.dist])
        if match.max == '':
            return random_value
        else:
            return (str(random_value) if int(match.max) > random_value else match.max)


test_instruction1 = '#vlo'
test_instruction2 = '#hi.5'

print(process_number(test_instruction1))
print(process_number(test_instruction2))